In [4]:
import pandas as pd
import os

# Import dataset from github
url = 'https://raw.githubusercontent.com/PlayingNumbers/ds_salary_proj/master/glassdoor_jobs.csv'
jobs = pd.read_csv(url, index_col=0)
#jobs.head()

In [14]:
# Looking in salary data
#jobs["Salary Estimate"].value_counts()

In [15]:
# cecking columns names
jobs.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')

In [22]:
# cleaning data for future usage

# separating hourely payed jobs and employer provided jobs

jobs['hourly'] = jobs['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
jobs['employer_provided'] = jobs['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

jobs = jobs[jobs['Salary Estimate'] != '-1']
salary = jobs['Salary Estimate'].apply(lambda x: x.split('(')[0])

# removing $ sign and K sign in salary description
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))
#print(minus_Kd.head())

# replacing per hour and employer provided salary 
min_hr = minus_Kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))
#print(min_hr)

# getting minimum maximum and average salaries
jobs['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
jobs['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))
jobs['avg_salary'] = (jobs.min_salary+jobs.max_salary)/2

#Company name text only
jobs['company_txt'] =jobs.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'][:-3], axis = 1)


#getting last 2 letters for state field 
jobs['job_state'] = jobs['Location'].apply(lambda x: x.split(',')[1])
#jobs.job_state.value_counts()

# checking if jobs are in Headquaters
jobs['same_state'] = jobs.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

#age of company  difference from todays date
jobs['age'] = jobs.Founded.apply(lambda x: x if x <1 else 2020 - x)

#parsing of job description (python, etc.)
#1. python
jobs['python_yn'] = jobs['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
 
#2. r studio 
jobs['R_yn'] = jobs['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
jobs.R_yn.value_counts()

#3. spark 
jobs['spark'] = jobs['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
jobs.spark.value_counts()

#4. aws 
jobs['aws'] = jobs['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
jobs.aws.value_counts()

#5. excel
jobs['excel'] = jobs['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
#jobs.excel.value_counts()


In [35]:
# checking new dataframe and columns

#jobs["company_txt"]
jobs.columns
jobs.to_csv('salary_data_cleaned.csv',index = False)